In [1]:
import pandas as pd
from os import walk
import numpy as np

In [2]:
def recode(filename):
    f = open(filename, 'r')
    f_c = open(filename + 'u', 'w')
    for line in f: 
        new_line = unicode(line, 'cp1251', 'ignore').encode('utf-8')
        f_c.write(new_line)
    f.close()
    f_c.close()

In [ ]:
recode('data/tolstoy/anna_carenina')
recode('data/tolstoy/kazaki')
recode('data/tolstoy/voskresenie')
recode('data/tolstoy_test/sev_stories')

Скачано несколько произведений указанных авторов, текст очищен от строк вроде "глава XII", "том II" и т.д.

In [23]:
# limits for text pieces sizes
from sklearn.utils import shuffle
import re
low = 30
high = 50

def clean_text(text):
    return u" ".join(re.findall(r'[\w]+', text.lower(), re.U))

def samples_from_file(filename, label):
    data = pd.DataFrame({'text' : [], 'label' : []})
    f = open(filename, 'r')
    size = np.random.randint(low = low, high = high)
    count = 0
    piece = ''
    for line in f:
        cl_line = clean_text(unicode(line, 'utf-8', 'ignore'))
        if len(cl_line) > 12:
            piece = piece + ' ' + cl_line
            count += 1
            if count >= size:
                count = 0   
                size = np.random.randint(low = low, high = high)
                addition = pd.DataFrame({'text' : [piece], 'label' : [label]})
                piece = ''
                data = pd.concat([data, addition])
    f.close()
    return data

Датасет для обучения

In [24]:
from os import walk
def scan_data(path, label):
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        for filename in filenames:
            f.append(samples_from_file(dirpath + '/' + filename, label))
    return pd.concat(f)
d1, d2 = scan_data('data/tolstoy', 1), scan_data('data/ilf_petrov', 0)
min_data = np.min([len(d1), len(d2)])
dataset = pd.concat([d1[:min_data], d2[:min_data]])
dataset.index = range(len(dataset))
dataset = shuffle(dataset)
dataset.to_csv('data/dataset.csv', encoding='utf-8')
print len(dataset)

576


Датасет для проверки. В них входят куски из произведений, не входящих в тренировочный датасет.

In [25]:
dataset = pd.concat([scan_data('data/tolstoy_test', 1), scan_data('data/ilf_petrov_test/', 0)])
# dataset = scan_data('data/tolstoy_test/', 1)
# dataset = scan_data('data/ilf_petrov_test/', 0)
dataset.index = range(len(dataset))
dataset = shuffle(dataset)
dataset.to_csv('data/test_dataset.csv', encoding='utf-8')
print len(dataset)

97


## Решение

Было скачано несколько произведений указанных авторов. Каждое было очищено от вспомогательных слов ("том 2", "глава X" и т.д.), знаков препинания, а так же переведено в нижний регистр. Далее из них формировалась выборка следующим образом: генерировалось случайное число n в определенных пределах, из произведения считывались n строк, и эти n строк считались элементом выборки. Для оценки качества предсказаний была использована кросс-валидация и проверка на произведениях, которые классификатор не видел при обучении.

In [26]:
dataset = pd.read_csv('data/dataset.csv', encoding='utf-8')
test_dataset = pd.read_csv('data/test_dataset.csv', encoding='utf-8')
all_data = pd.concat([dataset, test_dataset])

In [27]:
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
cv = TfidfVectorizer(min_df=10, binary=True, ngram_range=(1, 3)) 

X = cv.fit_transform(all_data.text)
X_train, X_test, y_train, y_test = X[:len(dataset)], X[len(dataset):], dataset.label.values, test_dataset.label.values

$n \in [10, 30]$

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

print 'cross_val_score: ', np.mean(cross_val_score(LogisticRegression(), X_train, y_train, cv=3))
clf = LogisticRegression()
clf.fit(X_train, y_train)
print 'test score: ', accuracy_score(y_test, clf.predict(X_test))

cross_val_score:  0.998268386586
test score:  0.989583333333


$n \in [10, 50]$

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

print 'cross_val_score: ', np.mean(cross_val_score(LogisticRegression(), X_train, y_train, cv=3))
clf = LogisticRegression()
clf.fit(X_train, y_train)
print 'test score: ', accuracy_score(y_test, clf.predict(X_test))

cross_val_score:  0.996153846154
test score:  1.0


$n \in [30, 50]$

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

print 'cross_val_score: ', np.mean(cross_val_score(LogisticRegression(), X_train, y_train, cv=3))
clf = LogisticRegression()
clf.fit(X_train, y_train)
print 'test score: ', accuracy_score(y_test, clf.predict(X_test))

cross_val_score:  1.0
test score:  1.0


## Выводы

1) регуляризация не дает никаких прибавок к качеству <br>
2) повышение размера n-граммов - тоже, похоже, основная разница кроется в словарном запасе <br>
3) последнюю догадку подтверждает и тот факт, что бинаризация признаков дает прибавку к качеству <br>
4) использование средней длины предложения на качество влияет отрицательно - похоже, длина предложений варьировалась в зависимости от периода жизни <br>
5) чем больше куски текста, на которых мы обучаемся и тестируем - тем выше точность. Для достаточно больших отрывков разделяющую плоскость можно провести точно.